In [1]:
%%sh
pip install pycaret
pip install python-dotenv
pip install ydata-profiling
pip install shap
pip -q install --upgrade stepfunctions
pip install s3fs

In [2]:
import boto3
import sagemaker
import time
import random
import uuid
import logging
import stepfunctions
import io
import random
import os

from sagemaker.amazon.amazon_estimator import get_image_uri
from stepfunctions import steps
from stepfunctions.steps import TrainingStep, ModelStep, TransformStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath


import os
from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data
from split_data import split_data
import importlib
from save_model_to_s3 import save_model_to_s3
from deploy_model_endpoint import deploy_model
from finalize_and_save_model import finalize_and_save_model
from delete_sagemaker_endpoint import delete_sagemaker_endpoint
# from ydata_profiling import ProfileReport
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = "s3://{}".format(data_location_s3)
instance_type = os.getenv("instance_type")
model_instance_count = int(os.getenv("model_instance_count"))
image_uri = os.getenv("ecr_repo_uri")
tuning_metric = os.getenv("tuning_metric")

print(
    data_location_s3,
    algorithm_choice,
    target,
    endpoint_name,
    model_name,
    data_location,
    instance_type,
    image_uri,
    tuning_metric,
)

streaming-data-platform-ml-data/ethan_data.csv classification y classification-proba-endpoint banking-classification s3://streaming-data-platform-ml-data/ethan_data.csv ml.m4.xlarge 135544376709.dkr.ecr.eu-west-1.amazonaws.com/mlops-classification-repo:latest AUC


In [4]:
stepfunctions.set_stream_logger(level=logging.INFO)
bucket = "streaming-data-platform-ml-data"
prefix = "step_function"
bucket_path = 's3://{}/{}/'.format(bucket, prefix)
region = boto3.Session().region_name
session = sagemaker.Session()

In [5]:
FILE_DATA = load_data(data_location)
print(FILE_DATA.head())

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


   age  job  education  default  balance  housing  loan  y
0   32    7          2        1     -238        1     0  0
1   34    4          2        0     -478        1     1  0
2   32    3          2        0      266        1     0  0
3   36    7          2        1       13        0     1  0
4   23   11          2        0      486        0     0  0


In [6]:
print(len(FILE_DATA))

55818


In [7]:
FILE_DATA = FILE_DATA.sample(frac=1)
FILE_DATA.head()

,age,job,education,default,balance,housing,loan,y
11073,30,2,2,0,222,1,0,0
21290,35,3,2,0,225,1,0,0
20574,62,7,2,0,180,0,0,1
16946,33,3,3,0,0,1,0,0
21561,53,3,4,0,9252,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

train_and_val_data, test_data = split_data(FILE_DATA, shuffle=True)

In [9]:
train_data, validation_data = split_data(train_and_val_data, shuffle=True)

In [10]:
print(len(train_data), len(validation_data), len(test_data))

35723 8931 11164


In [11]:
FILE_TRAIN = "train.csv"
FILE_TEST = "test.csv"
FILE_VALIDATION = "validation.csv"

In [12]:
train_s3_file = os.path.join(prefix, FILE_TRAIN)
test_s3_file = os.path.join(prefix, FILE_TEST)
validation_s3_file = os.path.join(prefix, FILE_VALIDATION)
print(train_s3_file, test_s3_file, validation_s3_file)

step_function/train.csv step_function/test.csv step_function/validation.csv


In [13]:
from io import StringIO

# Upload the three files to Amazon S3

csv_buffer = StringIO()
train_data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, train_s3_file).put(Body=csv_buffer.getvalue())

csv_buffer = StringIO()
test_data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, test_s3_file).put(Body=csv_buffer.getvalue())

csv_buffer = StringIO()
validation_data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, validation_s3_file).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'P34G3A3V44DPCC7M',
  'HostId': 'qfgR+LZtNqJF82qYBCuU1F/jPIS3hR5EJw+volCZ/EqEl54s4C3nlb4Q3AF93qRQDVpLkeYrvSM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qfgR+LZtNqJF82qYBCuU1F/jPIS3hR5EJw+volCZ/EqEl54s4C3nlb4Q3AF93qRQDVpLkeYrvSM=',
   'x-amz-request-id': 'P34G3A3V44DPCC7M',
   'date': 'Fri, 31 May 2024 14:46:50 GMT',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:eu-west-1:135544376709:key/e0f03833-4c3a-49bd-8c98-fa1223ee5a83',
   'etag': '"1f3581f5e7f58a2f02c214a2c0aec2af"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"1f3581f5e7f58a2f02c214a2c0aec2af"',
 'ServerSideEncryption': 'aws:kms',
 'SSEKMSKeyId': 'arn:aws:kms:eu-west-1:135544376709:key/e0f03833-4c3a-49bd-8c98-fa1223ee5a83'}

In [14]:
train_s3_file = 's3://{}/{}'.format(bucket, train_s3_file)
validation_s3_file = 's3://{}/{}'.format(bucket, validation_s3_file)
test_s3_file = 's3://{}/{}'.format(bucket, test_s3_file)
output_s3  = 's3://{}/{}/{}/'.format(bucket, prefix, 'output')
print(train_s3_file, validation_s3_file, test_s3_file, output_s3)

s3://streaming-data-platform-ml-data/step_function/train.csv s3://streaming-data-platform-ml-data/step_function/validation.csv s3://streaming-data-platform-ml-data/step_function/test.csv s3://streaming-data-platform-ml-data/step_function/output/


In [15]:
sagemaker_execution_role = role
xgb = sagemaker.estimator.Estimator(
    get_image_uri(region, 'xgboost', repo_version='0.90-2'),
    sagemaker_execution_role, 
    train_instance_count = 1, 
    train_instance_type = 'ml.m4.4xlarge',
    output_path = output_s3,
    sagemaker_session = session
)

xgb.set_hyperparameters(
    objective = 'reg:linear',
    num_round = 50,
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    silent = 0
)

In [16]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [17]:
training_step = steps.TrainingStep(
    'Train Step', 
    estimator=xgb,
    data={
        'train': sagemaker.inputs.TrainingInput(train_s3_file, content_type='libsvm'),
        'validation': sagemaker.inputs.TrainingInput(validation_s3_file, content_type='libsvm')
    },
    job_name=execution_input['JobName']
)

In [18]:
model_step = steps.ModelStep(
    'Save model',
    model=training_step.get_expected_model(),
    model_name=execution_input['ModelName']  
)

In [19]:
transform_step = steps.TransformStep(
    'Transform Input Dataset',
    transformer=xgb.transformer(
        instance_count=1,
        instance_type='ml.m5.large'
    ),
    job_name=execution_input['JobName'],     
    model_name=execution_input['ModelName'], 
    data=test_s3_file,
    content_type='text/libsvm'
)

In [20]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

In [21]:
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

In [22]:
workflow_definition = steps.Chain([
    training_step,
    model_step,
    transform_step,
    endpoint_config_step,
    endpoint_step
])

In [23]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

workflow = Workflow(
    name='{}-{}'.format('MyTrainTransformDeploy_v1', timestamp),
    definition=workflow_definition,
    role=role,
    execution_input=execution_input
)

In [24]:
workflow.render_graph()

In [25]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:eu-west-1:135544376709:stateMachine:MyTrainTransformDeploy_v1-31-14-46-50'

In [26]:
execution = workflow.execute(
    inputs={
        'JobName': 'regression-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'regression-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'regression-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


In [28]:
execution.render_progress()